In [ ]:
!pip install simpletransformers -q


import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

     |████████████████████████████████| 225kB 8.3MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 7.8MB 24.1MB/s 
     |████████████████████████████████| 3.3MB 47.9MB/s 
     |████████████████████████████████| 2.5MB 47.7MB/s 
     |████████████████████████████████| 122kB 45.4MB/s 
     |████████████████████████████████| 245kB 57.8MB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 1.2MB 46.5MB/s 
     |████████████████████████████████| 4.2MB 45.8MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 112kB 60.7MB/s 
     |████████████████████████████████| 901kB 35.6MB/s 
     |████████████████████████████████| 122kB 57.2MB/s 
     |████████████████████████████████| 245kB 35.0MB/s 
     |████████████████████████████████| 122kB 60.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompati

In [ ]:
import torch
import numpy as np
import os
import random

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(7)

In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
#For logging loss
wandb.login(key="INSERT YOUR API KEY")

In [ ]:
import os
import pandas as pd

PATH_TO_DATASET = "../input/yoruba-translation/"  #Where you stored the dataset

train = pd.read_csv(os.path.join(PATH_TO_DATASET, "Train.csv"))

#Remove any possible duplicates
train = train.drop_duplicates(subset=["Yoruba", "English"])

#Lowercase and remove trailing spaces
train["Yoruba"] = train.apply(lambda x: (x.Yoruba).strip().lower(), axis=1)
train["English"] = train.English.apply(lambda x: x.lower())

train = train[["Yoruba", "English"]]
train.columns = ["input_text", "target_text"]

#Train 95% / Validation 5% Split
validation = train.sample(frac=0.05).astype(str)
train = train.drop(index=validation.index).astype(str)

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_df = train
eval_df = validation

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 100
model_args.train_batch_size = 10
model_args.eval_batch_size = 10
model_args.num_train_epochs = 8
model_args.scheduler = "cosine_schedule_with_warmup"
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 10000
model_args.learning_rate = 0.0003
model_args.optimizer = 'Adafactor'
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.save_model_every_epoch = False
model_args.preprocess_inputs = False
model_args.use_early_stopping = True
model_args.num_return_sequences = 1
model_args.do_lower_case = True
model_args.output_dir = "all/"
model_args.best_model_dir = "all/best_model"
model_args.wandb_project = "Yoruba fine tune"

model = T5Model("mt5", "google/mt5-base", args=model_args)
#model.model.load_state_dict(torch.load("../input/semifinalyoruba/outputs/best_model/pytorch_model.bin"))

# Train the model
model.train_model(train_df, eval_data=eval_df)

# Optional: Evaluate the model. We'll test it properly anyway.
results = model.eval_model(eval_df, verbose=True)

In [ ]:
model_args = T5Args()
model_args.max_length = 100
model_args.length_penalty = 2.5
model_args.repetition_penalty = 1.5
model_args.num_beams = 5

model = T5Model("mt5", "all/best_model", args=model_args)

In [ ]:
preds = model.predict(validation.input_text.values.tolist())

In [ ]:
validation["preds"] = preds